# PART III: Feature Engineering & Enrichment

## 1. Executive Summary
Following the data cleaning in Part II, this phase transforms the raw dataset into a machine-learning-ready format. Our approach goes beyond simple formatting; we integrate **domain knowledge** and **external data sources** to construct high-value features.

**Key Highlights of Feature Engineering:**
* **Geospatial Enrichment**: Integrated external government data (MRT stations & Malls) via API to quantify "connectivity" and "lifestyle convenience".
* **Handling "Cold Start"**: Created `is_new_listing` flags to handle new properties with missing reviews, preserving their "newness" signal while enabling mathematical modeling.
* **Text & Temporal Mining**: Extracted quantitative metrics from unstructured text (descriptions) and dates (host tenure).
* **Dimensionality Expansion**: Applied One-Hot Encoding to categorical variables, expanding the dataset to **125+ numerical features** ready for direct modeling.

---

In [1]:
import pandas as pd
import numpy as np
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load the cleaned dataset from Part II
input_path = '../data/listings_cleaned.csv'
df = pd.read_csv(input_path)
print(f"Data Loaded from {input_path}. Shape: {df.shape}")

Data Loaded from ../data/listings_cleaned.csv. Shape: (3693, 78)


## 2. Advanced Preprocessing & Imputation Strategy

### 2.1 Handling Systematic Missingness (Review Scores)
A significant portion of listings (~50%) have missing `review_scores` not because of data errors, but because they are **new listings with zero reviews**. Dropping them would bias the model against new inventory.

**Our Strategy:**
1.  **Signal Preservation (`is_new_listing`)**: We created a binary feature `is_new_listing` (1 = New/No Reviews, 0 = Established). This allows the model to treat new listings differently from bad listings.
2.  **Median Imputation**: We filled the missing scores with the **median value** of the dataset. This neutralizes the missing values mathematically, allowing algorithms to function without crashing, while the `is_new_listing` flag handles the actual "newness" effect.

### 2.2 Final Cleanup
We removed residual ID columns (e.g., `listing_url`, `host_thumbnail_url`) and text-heavy columns that were flagged as "Governance/Privacy Risks" or "Noise" in Part II.

In [2]:
# 1. Convert Date Columns
date_cols = ['host_since', 'first_review', 'last_review']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# 2. Cleanup Residual Garbage Columns (if any remain from Part II)
cols_to_drop = [
    'listing_url', 'scrape_id', 'last_scraped', 'source', 'picture_url', 
    'host_url', 'host_thumbnail_url', 'host_picture_url', 'calendar_last_scraped',
    'neighbourhood', 'host_neighbourhood', 'host_location', 'host_about', 
    'host_name', 'host_verifications', 'license', 'neighborhood_overview'
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# 3. Handle Systematic Missing Values (Review Scores)
# Create a flag for listings with no reviews
if 'review_scores_rating' in df.columns:
    df['is_new_listing'] = df['review_scores_rating'].isnull().astype(int)

# Impute missing scores with the median
review_cols = [c for c in df.columns if 'review_scores' in c]
for col in review_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

print("Preprocessing complete. Missing review scores imputed.")

Preprocessing complete. Missing review scores imputed.


## 3. Feature Construction & Enrichment

We engineered new features across three dimensions: **Text**, **Space**, and **Time**.

### 3.1 Text Mining: Quantifying "Luxury" & "Effort"
Raw text fields cannot be used directly in regression models. We extracted quantitative proxies:
* **`amenities_count`**: Total number of amenities provided. (Hypothesis: More amenities = Higher Price).
* **`has_pool`, `has_gym`, `has_ac`...**: Binary flags for high-value keywords extracted from the amenities list.
* **`description_length` / `name_length`**: Character counts used as a proxy for **Host Effort** and listing detail.



In [3]:
def count_amenities(x):
    try:
        # Clean and count the items in the list string
        return len(x.replace('[','').replace(']','').replace('"','').split(','))
    except:
        return 0

if 'amenities' in df.columns:
    # Feature 1: Count
    df['amenities_count'] = df['amenities'].apply(count_amenities)
    
    # Feature 2: Key Amenities Flags
    target_amenities = ['Air conditioning', 'Pool', 'Gym', 'Wifi', 'Kitchen', 'Washer']
    for amenity in target_amenities:
        col_name = f'has_{amenity.lower().replace(" ", "_")}'
        df[col_name] = df['amenities'].str.contains(amenity, case=False, regex=False).astype(int)
    
    # Drop original column
    df = df.drop(columns=['amenities'])
    
print("Amenities features created.")

Amenities features created.


### 3.2 Geospatial and Temporal Features
- **Distance to Center**: Calculated using the Haversine formula relative to Marina Bay Sands (City Center).
- **Host Tenure**: Calculated as the number of years the host has been active.
- **Name/Description Length**: Extracted as proxies for host effort and listing detail.

In [4]:
# 1. Geospatial: Distance to Marina Bay Sands (1.2834, 103.8607)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

df['distance_to_center_km'] = haversine_distance(df['latitude'], df['longitude'], 1.2834, 103.8607)

# 2. Temporal: Host Tenure (Years)
ref_date = df['last_review'].max() # Reference date
if 'host_since' in df.columns:
    df['host_years'] = (ref_date - df['host_since']).dt.days / 365.0
    df['host_years'] = df['host_years'].fillna(0)

# 3. Text Length Features
if 'name' in df.columns:
    df['name_length'] = df['name'].fillna('').astype(str).apply(len)
if 'description' in df.columns:
    df['description_length'] = df['description'].fillna('').astype(str).apply(len)

print("Geospatial and temporal features created.")

Geospatial and temporal features created.


### 3.3 Geospatial Enrichment: MRT & Malls Proximity Analysis (New Feature)
To capture the critical impact of location on property value, we extended the dataset with **external geospatial data** covering both transport and lifestyle amenities.

**Methodology:**
1.  **Data Acquisition**: We compiled geospatial data for all **Singapore MRT Stations** (Transport Connectivity) and **Major Shopping Malls** (Lifestyle Convenience).
2.  **Geocoding**: Precise coordinates (Latitude/Longitude) were retrieved via the **Singapore Government OneMap API**.
3.  **Feature Construction**: Using the Haversine formula, we calculated the **minimum distance** from each listing to the nearest MRT station and the nearest Shopping Mall.

In [5]:
# ==========================================
# NEW FEATURE: Proximity to Amenities (MRT & Malls)
# ==========================================

import pandas as pd
import numpy as np

# 1. Load External Data
# Note: Ensure both 'mrt_stations.csv' and 'shopping_malls.csv' are in the '../data/' directory
try:
    mrt_path = '../data/mrt_stations.csv'
    mall_path = '../data/shopping_malls.csv' # 假设你的商场数据文件名叫这个
    
    mrt_df = pd.read_csv(mrt_path)
    mall_df = pd.read_csv(mall_path)
    
    print(f"External Data Loaded:")
    print(f"- MRT Stations: {len(mrt_df)}")
    print(f"- Shopping Malls: {len(mall_df)}")

    # 2. Define Vectorized Haversine Function (Generic for any target points)
    def get_nearest_distance(row, target_df):
        R = 6371  # Earth radius in km
        
        # Convert degrees to radians
        lat1, lon1 = np.radians(row['latitude']), np.radians(row['longitude'])
        lat2 = np.radians(target_df['latitude'].values)
        lon2 = np.radians(target_df['longitude'].values)
        
        # Haversine formula
        dphi = lat2 - lat1
        dlambda = lon2 - lon1
        a = np.sin(dphi/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlambda/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        distances = R * c
        
        # Return the minimum distance
        return distances.min()

    # 3. Apply Calculation for MRT
    print("Calculating distance to nearest MRT... (Step 1/2)")
    df['dist_to_mrt_km'] = df.apply(lambda x: get_nearest_distance(x, mrt_df), axis=1)

    # 4. Apply Calculation for Malls
    print("Calculating distance to nearest Mall... (Step 2/2)")
    df['dist_to_mall_km'] = df.apply(lambda x: get_nearest_distance(x, mall_df), axis=1)
    
    print("✅ Geospatial Features created: 'dist_to_mrt_km', 'dist_to_mall_km'")

except FileNotFoundError as e:
    print(f"⚠️ Warning: External data file not found. Details: {e}")
    # Create placeholders to prevent code failure
    if 'dist_to_mrt_km' not in df.columns: df['dist_to_mrt_km'] = -1
    if 'dist_to_mall_km' not in df.columns: df['dist_to_mall_km'] = -1

except Exception as e:
    print(f"⚠️ Error during proximity calculation: {e}")

External Data Loaded:
- MRT Stations: 213
- Shopping Malls: 22
Calculating distance to nearest MRT... (Step 1/2)
Calculating distance to nearest Mall... (Step 2/2)
✅ Geospatial Features created: 'dist_to_mrt_km', 'dist_to_mall_km'


## 4. Categorical Encoding & Output Generation

### 4.1 Encoding Strategy
* **Ordinal Encoding**: `host_response_time` was mapped to an ordered scale (4=Within an hour ... 1=Few days), reflecting the value of responsiveness.
* **One-Hot Encoding**: Nominal variables like `room_type`, `neighbourhood_group`, and `property_type` were expanded into binary columns (0/1).
    * *Note* : This expansion explains the increase in column count (from ~78 to ~125).
* **Final Cleanup**: All remaining non-numeric columns are dropped to ensure the dataset is compatible with modeling algorithms.

In [6]:
# 1. Ordinal Encoding for Host Response Time
response_map = {
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}
if 'host_response_time' in df.columns:
    df['host_response_score'] = df['host_response_time'].map(response_map).fillna(0)

# 2. Boolean Handling (ensure 1/0)
bool_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable']
for col in bool_cols:
    if col in df.columns:
        if df[col].dtype == 'object':
             df[col] = df[col].map({'t': 1, 'f': 0}).fillna(0)
        else:
             df[col] = df[col].astype(int)

# 3. One-Hot Encoding
categorical_cols = ['room_type', 'neighbourhood_group_cleansed', 'property_type']
existing_cats = [c for c in categorical_cols if c in df.columns]
df = pd.get_dummies(df, columns=existing_cats, drop_first=True)

# 4. Final Drop of Processed/Text Columns
cols_to_remove = ['name', 'description', 'host_response_time', 'host_since', 'first_review', 'last_review', 'bathrooms_text']
df = df.drop(columns=[c for c in cols_to_remove if c in df.columns], errors='ignore')

# 5. Sanity Check: Drop any remaining object columns
non_numeric = df.select_dtypes(include=['object']).columns
if len(non_numeric) > 0:
    print(f"Dropping remaining non-numeric columns: {list(non_numeric)}")
    df = df.drop(columns=non_numeric)

# Export
output_path = '../data/listings_featured.csv'
df.to_csv(output_path, index=False)

print(f"Feature Engineering Complete. Final Shape: {df.shape}")
print(f"File saved to: {output_path}")

Dropping remaining non-numeric columns: ['neighbourhood_cleansed']
Feature Engineering Complete. Final Shape: (3693, 125)
File saved to: ../data/listings_featured.csv


### 4.2 Final Deliverable (`listings_featured.csv`)
The output file is a **strictly numerical matrix** optimized for Scikit-Learn/XGBoost.

**Data Dictionary for New Features:**
* `is_new_listing` (int): 1 if the listing has no reviews, 0 otherwise.
* `dist_to_mrt_km` (float): Distance to the nearest MRT station in km.
* `dist_to_mall_km` (float): Distance to the nearest Shopping Mall in km.
* `amenities_count` (int): Total number of amenities.
* `host_years` (float): Host tenure in years.
* `host_response_score` (int): 1 (Slow) to 4 (Fast).